In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pyarrow as pa
from datetime import datetime

In [3]:
def gestionDonneesAberrantes(dataGroup, variableStr, minTheorique, maxTheorique, columnStr):
    
    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath = dataPath + "rawData/" + encounterId + "/" + variableStr + ".csv"
        df = pd.read_csv(dfPath)
        
        df_filtered_nan = df[~pd.isna(df[columnStr])]

        df_filtered = df_filtered_nan[(df_filtered_nan[columnStr] >= minTheorique) & (df_filtered_nan[columnStr] <= maxTheorique)]

        newDfPath = dataPath + "preProcessedData/" + encounterId + "/" + variableStr + ".parquet"
        pq.write_table(pa.Table.from_pandas(df_filtered), newDfPath)

In [53]:
def gestionCompliance():

    dataPath = "../dataMimic/"
    
    patients_df = pd.read_csv(dataPath + "ventiles1.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)
    c = 0
    for _, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        df_PEP_Path = preProcessedDataPath + encounterId + "/PEP.parquet"
        df_Pplat_Path = preProcessedDataPath + encounterId + "/Pplat.parquet"
        df_VolumeCourant_Path = preProcessedDataPath + encounterId + "/VolumeCourant.parquet"

        df_PEP = pd.read_parquet(df_PEP_Path)
        df_Pplat = pd.read_parquet(df_Pplat_Path)
        df_VolumeCourant = pd.read_parquet(df_VolumeCourant_Path)
        
        installation_date = pd.Timestamp(row["start_vent"])
        withdrawal_date = installation_date + pd.Timedelta(days=row["vent_lenght"])
        total_time_hour = (withdrawal_date - installation_date).total_seconds() / 3600 + 4

        liste_valeurs = []
        liste_temps = []        
        
        for i in range(int(total_time_hour)):

            current_time_minute = i*60
            
            VolumeCourant_current_values = df_VolumeCourant[df_VolumeCourant["time"]==current_time_minute]
            Pplat_current_values = df_Pplat[df_Pplat["time"]==current_time_minute]
            PEP_current_values = df_PEP[df_PEP["time"]==current_time_minute]

            if VolumeCourant_current_values.size > 0 and Pplat_current_values.size > 0 and PEP_current_values.size > 0:
                
                volumeCourant = VolumeCourant_current_values["VolumeCourant"].iloc[0]
                Pplat = Pplat_current_values["Pplat"].iloc[0]
                PEP = PEP_current_values["PEP"].iloc[0]
                
                compliance = volumeCourant / (Pplat - PEP)
                # print(f"VC: {volumeCourant}, Pplat: {Pplat} with {prio}, PEP: {PEP}, Compliance: {compliance}")

                if compliance > 0 and compliance < 150:
                    liste_valeurs.append(compliance)
                    liste_temps.append(current_time_minute)
        
        # if c==13:
        #     return liste_valeurs, liste_temps
        # c+=1
        newdf = pd.DataFrame({'Compliance': liste_valeurs, 'temps': liste_temps})
        newdfPath = preProcessedDataPath + encounterId + "/Compliance_Moy.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newdfPath)

In [54]:
gestionCompliance()

  1%|▊                                                                                                                                                           | 26/4657 [00:04<16:15,  4.75it/s]/tmp/ipykernel_124169/4157913798.py:43: RuntimeWarning: divide by zero encountered in scalar divide
  compliance = volumeCourant / (Pplat - PEP)
  6%|████████▋                                                                                                                                                  | 261/4657 [00:43<12:58,  5.64it/s]/tmp/ipykernel_124169/4157913798.py:43: RuntimeWarning: divide by zero encountered in scalar divide
  compliance = volumeCourant / (Pplat - PEP)
  9%|█████████████▌                                                                                                                                             | 409/4657 [01:09<10:14,  6.91it/s]/tmp/ipykernel_124169/4157913798.py:43: RuntimeWarning: divide by zero encountered in scalar divide
  compliance = volumeCour

In [16]:
def gestionDiurese():

    dataPath = "../dataMimic/"
    
    patients_df = pd.read_csv(dataPath + "ventiles1.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    rawDataPath = dataPath + "rawData/"
    
    nb_patients = len(patients_df)
    c = 0
    for _, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        df_Dialyse_Path = rawDataPath + encounterId + "/dialyse.csv"

        list_Dialyse = pd.read_csv(df_Dialyse_Path).to_numpy().tolist()

        installation_date = pd.Timestamp(row["start_vent"])
        withdrawal_date = installation_date + pd.Timedelta(days=row["vent_lenght"])
        total_time_hour = (withdrawal_date - installation_date).total_seconds() / 3600 + 4
        
        liste_valeurs = [0] * int(total_time_hour)
        
        for e in list_Dialyse:
            
            temps = e[0]
            current_time_hour = (temps+30)//60

            if current_time_hour+1 < int(total_time_hour) and current_time_hour > 0:
                liste_valeurs[current_time_hour] = 1
                liste_valeurs[current_time_hour+1] = 1
    
            
        newdf = pd.DataFrame({'Dialyse': liste_valeurs})
        newdfPath = preProcessedDataPath + encounterId + "/Dialyse_Complet.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newdfPath)

        newdf = pd.DataFrame({'Dialyse': np.array(liste_valeurs)*0+1})
        newdfPath = preProcessedDataPath + encounterId + "/Dialyse_Mask.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newdfPath)

In [17]:
gestionDiurese()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4657/4657 [00:07<00:00, 604.83it/s]


In [4]:
def gestionPoids(dataGroup):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)
    
    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath = preProcessedDataPath + encounterId + "/Weight.parquet"
        
        df = pd.read_parquet(dfPath)
        
        numeric_values = pd.to_numeric(df.iloc[:, 0], errors='coerce').dropna()

        if df.size == 0:
            meanWeight = 80.0001
        else:
            meanWeight = np.mean(numeric_values)

        newdf = pd.DataFrame([meanWeight])

        newDfPath = preProcessedDataPath + encounterId + "/Weight3.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newDfPath)


def gestionTaille(dataGroup):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath = preProcessedDataPath + encounterId + "/Height.parquet"

        df = pd.read_parquet(dfPath)
        
        numeric_values = pd.to_numeric(df.iloc[:, 0], errors='coerce').dropna()

        if df.size == 0:
            meanHeight = 171.0001
        else:
            meanHeight = np.mean(numeric_values)
        
        newdf = pd.DataFrame([meanHeight])

        newDfPath = preProcessedDataPath + encounterId + "/Height_Moy.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newDfPath)


def gestionIMC(dataGroup):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    gestionPoids(dataGroup)
    gestionTaille(dataGroup)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath_height = preProcessedDataPath + encounterId + "/Height_Moy.parquet"
        height = pd.read_parquet(dfPath_height).iloc[0][0]

        dfPath_weight = preProcessedDataPath + encounterId + "/Weight3.parquet"
        weight = pd.read_parquet(dfPath_weight).iloc[0][0]

        imc = weight / ((height/100)**2)

        newdf = pd.DataFrame([imc])
        newDfPath = preProcessedDataPath + encounterId + "/IMC.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newDfPath)


def gestionDiurese(dataGroup, h_for_avg):
    
    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath = preProcessedDataPath + encounterId + "/Diurese.parquet"
        
        df = pd.read_parquet(dfPath)
        
        weight = pd.read_parquet(preProcessedDataPath+encounterId+"/Weight3.parquet").to_numpy()[0][0]

        liste_diurese_moy = []
        liste_temps = []
        
        for i in range(h_for_avg,df['time'].size):
            temps_i = df['time'].iloc[i]
            valide = True
            diurese_moy = 0
            for j in range(h_for_avg):
                if df['time'].iloc[i-j] == (temps_i - 60*j):
                    diurese_moy += df['Diurese'].iloc[i-j]
                else:
                    valide = False
                    break
            diurese_moy = diurese_moy/h_for_avg/weight
            if valide and temps_i > -h_for_avg*60:
                liste_diurese_moy.append(diurese_moy)
                liste_temps.append(temps_i)

        newdf = pd.DataFrame({'Diurese': liste_diurese_moy, 'time': liste_temps})

        newDfPath = preProcessedDataPath + encounterId + "/Diurese0.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newDfPath)


def gestionFiO2(dataGroup, frequenceAcquisition):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    columnValuesStr_FiO2 = 'FiO2'
    columnValuesStr_SpO2 = 'SpO2'

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath_FiO2 = preProcessedDataPath + encounterId + "/FiO2.parquet"
        dfPath_SpO2 = preProcessedDataPath + encounterId + "/SpO2_Moy.parquet"

        df_FiO2 = pd.read_parquet(dfPath_FiO2)
        df_SpO2 = pd.read_parquet(dfPath_SpO2)

        sizeDf_FiO2 = df_FiO2[columnValuesStr_FiO2].size
        sizeDf_SpO2 = df_SpO2[columnValuesStr_SpO2].size
        new_index = range(sizeDf_FiO2)
        df_FiO2.index = new_index

        liste_valeurs = []
        
        # Test si le df est vide
        if sizeDf_FiO2 > 0:
            lastTime = int(df_FiO2['time'].iloc[-1] // frequenceAcquisition)
        else:
            lastTime = 0

        nbValeurs = df_FiO2.index.max()+1

        # On calcule la valeur moyenne de la variable d'intérêt sur un intervalle de temps donné en paramètre
        index_current_time = 0
        for i in range(lastTime):
            FiO2_sum = 0
            current_time = df_FiO2['time'][index_current_time]
            compteur = 0

            while (current_time < 0):
                index_current_time += 1
                current_time = df_FiO2['time'][index_current_time]
            
            while (current_time < (i+1)*frequenceAcquisition) and (index_current_time < nbValeurs):
                FiO2_sum += df_FiO2[columnValuesStr_FiO2][index_current_time] 

                index_current_time += 1
                current_time = df_FiO2['time'][index_current_time]
                compteur += 1
            
            if compteur > 0:
                FiO2_moy = FiO2_sum/compteur
                if i<sizeDf_SpO2 and not(np.isnan(df_SpO2[columnValuesStr_SpO2][i])):
                    liste_valeurs.append(df_SpO2[columnValuesStr_SpO2][i]/FiO2_moy)
                else:
                    liste_valeurs.append(np.nan)
            else:
                liste_valeurs.append(np.nan)

        
        liste_temps = list(range(0,lastTime*frequenceAcquisition,frequenceAcquisition))

        newdf = pd.DataFrame({'SpO2_sur_FiO2': liste_valeurs, 'temps': liste_temps})

        newDfPath = preProcessedDataPath + encounterId + "/SpO2_sur_FiO2_Moy.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newDfPath)


def moyenne_sur_x_minutes(dataGroup, variableStr, frequenceAcquisition, columnValuesStr):
    
    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        dfPath = preProcessedDataPath + encounterId + "/" + variableStr + ".parquet"
        
        df = pd.read_parquet(dfPath)

        sizeDf = df[columnValuesStr].size
        new_index = range(sizeDf)
        df.index = new_index

        liste_valeurs = []
        
        # Test si le df est vide
        if sizeDf > 0:
            lastTime = int(df['time'].iloc[-1] // frequenceAcquisition)
        else:
            lastTime = 0

        nbValeurs = df.index.max()+1

        # On calcule la valeur moyenne de la variable d'intérêt sur un intervalle de temps donné en paramètre
        index_current_time = 0
        for i in range(lastTime):
            valeur_moy = 0
            current_time = df['time'][index_current_time]
            compteur = 0
            
            while (current_time < 0):
                index_current_time += 1
                current_time = df['time'][index_current_time]
                
            while (current_time < (i+1)*frequenceAcquisition) and (index_current_time < nbValeurs):
                valeur_moy += df[columnValuesStr][index_current_time] 

                index_current_time += 1
                current_time = df['time'][index_current_time]
                compteur += 1
            
            if compteur > 0:
                liste_valeurs.append(valeur_moy/compteur)
            else:
                liste_valeurs.append(np.nan)

        
        liste_temps = list(range(0,lastTime*frequenceAcquisition,frequenceAcquisition))

        newdf = pd.DataFrame({columnValuesStr: liste_valeurs, 'temps': liste_temps})

        newDfPath = preProcessedDataPath + encounterId + "/" + variableStr + "_Moy.parquet"
        pq.write_table(pa.Table.from_pandas(newdf), newDfPath)

In [4]:
def gestionValeursManquantes(dataGroup, variableStr, columnValuesStr, defaultValue, isDiurese=False):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles1.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

            encounterId = str(row["stay_id"])

            dfPath = preProcessedDataPath + encounterId + "/" + variableStr + "_Moy" + ".parquet"
            
            df = pd.read_parquet(dfPath)

            sizeDf = df[columnValuesStr].size

            # La taille des listes est fixée au nombre d'heures total du séjour (+4) 
            installation_date = pd.Timestamp(row["start_vent"])
            withdrawal_date = installation_date + pd.Timedelta(days=row["vent_lenght"])
            total_time_hour = (withdrawal_date - installation_date).total_seconds() / 3600 + 4

            liste_valeurs = []
            liste_mask = [] # permet de savoir si une valeur est manquante à l'origine
            
            if df[columnValuesStr].isnull().all() or sizeDf == 0:
                liste_valeurs = [defaultValue] * int(total_time_hour)
                liste_mask = [0] * int(total_time_hour)
                sizeDf = 0
            
            if sizeDf > 0:
                for i in range(int(total_time_hour)):

                    if i >= sizeDf: # plus de valeurs existantes à partir de là
                        val_i = np.nan
                    else:
                        val_i = df[columnValuesStr][i]

                    if np.isnan(val_i):
                        
                        liste_mask.append(0) # On crée une valeur qui n'existe pas
                        
                        j = 1
                        found = False
                        
                        # On associe la valeur la plus proche
                        # En cas d'equidistance la moyenne entre les 2 valeurs les plus proches
                        while not(found):
                            if i-j < 0:
                                while i+j < sizeDf:
                                    val_i_plus_j = df[columnValuesStr][i+j]
                                    if not(np.isnan(val_i_plus_j)):
                                        new_val = val_i_plus_j
                                        found = True
                                        break
                                    j += 1
                            elif i+j >= sizeDf:
                                while i-j >= sizeDf:
                                    j += 1
                                while i-j >= 0:
                                    val_i_moins_j = df[columnValuesStr][i-j]
                                    if not(np.isnan(val_i_moins_j)):
                                        new_val = val_i_moins_j
                                        found = True
                                        break
                                    j += 1
                            else:
                                val_i_moins_j = df[columnValuesStr][i-j]
                                val_i_plus_j = df[columnValuesStr][i+j]
                                if not(np.isnan(val_i_moins_j)):
                                    if not(np.isnan(val_i_plus_j)):
                                        new_val = (val_i_moins_j + val_i_plus_j) * 0.5
                                    else:
                                        new_val = val_i_moins_j
                                    found = True
                                elif not(np.isnan(val_i_plus_j)):
                                    new_val = val_i_plus_j
                                    found = True
                            j += 1
                        
                        liste_valeurs.append(new_val)

                    else:
                        liste_mask.append(1) # La valeur existe
                        liste_valeurs.append(val_i)
            
            if isDiurese:
                newdf = pd.DataFrame({columnValuesStr: liste_valeurs})
                newdfPath = preProcessedDataPath + encounterId + "/Diurese_Complet.parquet"
                pq.write_table(pa.Table.from_pandas(newdf), newdfPath)
    
                newdfMask = pd.DataFrame({'mask': liste_mask})
                newdfMaskPath = preProcessedDataPath + encounterId + "/Diurese_Mask.parquet"
                pq.write_table(pa.Table.from_pandas(newdfMask), newdfMaskPath)
            else:
                newdf = pd.DataFrame({columnValuesStr: liste_valeurs})
                newdfPath = preProcessedDataPath + encounterId + "/" + variableStr + "_Complet.parquet"
                pq.write_table(pa.Table.from_pandas(newdf), newdfPath)
    
                newdfMask = pd.DataFrame({'mask': liste_mask})
                newdfMaskPath = preProcessedDataPath + encounterId + "/" + variableStr + "_Mask.parquet"
                pq.write_table(pa.Table.from_pandas(newdfMask), newdfMaskPath)



# Fonction à part pour les pressions artérielles
# On associe d'abord la PAI si elle existe, sinon la PANI si elle existe, sinon la PAI la plus proche si elle existe, sinon la PANI la plus proche
def gestionValeursManquantesPA(dataGroup, PA_Str, columnValuesStr, defaultValue):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    PA_I_Str = PA_Str + "_I"
    PA_NI_str = PA_Str + "_NI"

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):
            
            columnValuesStr_PAI = columnValuesStr + "_I"
            columnValuesStr_PANI = columnValuesStr + "_NI"

            encounterId = str(row["stay_id"])

            dfPath = preProcessedDataPath + encounterId + "/" + PA_I_Str + "_Moy" + ".parquet"
            dfPath2 = preProcessedDataPath + encounterId + "/" + PA_NI_str + "_Moy" + ".parquet"

            df = pd.read_parquet(dfPath)
            df2 = pd.read_parquet(dfPath2)

            sizeDf = df[columnValuesStr_PAI].size
            sizeDf2 = df2[columnValuesStr_PANI].size

            installation_date = pd.Timestamp(row["start_vent"])
            withdrawal_date = installation_date + pd.Timedelta(days=row["vent_lenght"])
            total_time_hour = (withdrawal_date - installation_date).total_seconds() / 3600 + 4

            liste_valeurs = []
            liste_mask = [] # permet de savoir si une valeur est manquante à l'origine

            if df[columnValuesStr_PAI].isnull().all() or sizeDf == 0:
                if df2[columnValuesStr_PANI].isnull().all() or sizeDf2 == 0:
                    liste_valeurs = [defaultValue] * int(total_time_hour)
                    liste_mask = [0] * int(total_time_hour)
                    sizeDf = 0
                else:
                    df = df2
                    columnValuesStr_PAI = columnValuesStr_PANI
                    sizeDf = sizeDf2

            if sizeDf > 0:
                for i in range(int(total_time_hour)):

                    if i >= sizeDf:
                        val_i = np.nan
                    else:
                        val_i = df[columnValuesStr_PAI][i]

                    if np.isnan(val_i):

                        if (i < sizeDf2) and (not(np.isnan(df2[columnValuesStr_PANI][i]))):
                            new_val = df2[columnValuesStr_PANI][i]
                            liste_valeurs.append(new_val)
                            liste_mask.append(1)
                        
                        else:
                            
                            liste_mask.append(0)

                            j = 1
                            found = False
                            
                            while not(found):
                                if i-j < 0:
                                    while i+j < sizeDf:
                                        val_i_plus_j = df[columnValuesStr_PAI][i+j]
                                        if not(np.isnan(val_i_plus_j)):
                                            new_val = val_i_plus_j
                                            found = True
                                            break
                                        j += 1
                                elif i+j >= sizeDf:
                                    while i-j >= sizeDf:
                                        j += 1
                                    while i-j >= 0:
                                        val_i_moins_j = df[columnValuesStr_PAI][i-j]
                                        if not(np.isnan(val_i_moins_j)):
                                            new_val = val_i_moins_j
                                            found = True
                                            break
                                        j += 1
                                else:
                                    val_i_moins_j = df[columnValuesStr_PAI][i-j]
                                    val_i_plus_j = df[columnValuesStr_PAI][i+j]
                                    if not(np.isnan(val_i_moins_j)):
                                        if not(np.isnan(val_i_plus_j)):
                                            new_val = (val_i_moins_j + val_i_plus_j) * 0.5
                                        else:
                                            new_val = val_i_moins_j
                                        found = True
                                    elif not(np.isnan(val_i_plus_j)):
                                        new_val = val_i_plus_j
                                        found = True
                                j += 1
                            
                            liste_valeurs.append(new_val)

                    else:
                        liste_mask.append(1)
                        liste_valeurs.append(val_i)
            

            newdf = pd.DataFrame({columnValuesStr: liste_valeurs})
            newDfPath = preProcessedDataPath + encounterId + "/" + columnValuesStr.upper() + "_Complet.parquet"
            pq.write_table(pa.Table.from_pandas(newdf), newDfPath)

            newdfMask = pd.DataFrame({'mask': liste_mask})
            newdfMaskPath = preProcessedDataPath + encounterId + "/" + columnValuesStr.upper() + "_Mask.parquet"
            pq.write_table(pa.Table.from_pandas(newdfMask), newdfMaskPath)



# Fonction à part pour les températures
def gestionValeursManquantesTemperatures(columnValuesStr, defaultValue):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles1.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    nb_patients = len(patients_df)

    PA_I_Str = "Temperature_C"
    PA_NI_str = "Temperature_F"

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):
            
            columnValuesStr_PAI = PA_I_Str
            columnValuesStr_PANI = PA_NI_str

            encounterId = str(row["stay_id"])

            dfPath = preProcessedDataPath + encounterId + "/" + PA_I_Str + "_Moy" + ".parquet"
            dfPath2 = preProcessedDataPath + encounterId + "/" + PA_NI_str + "_Moy" + ".parquet"

            df = pd.read_parquet(dfPath)
            df2 = (pd.read_parquet(dfPath2) - 32) / 1.8

            sizeDf = df[columnValuesStr_PAI].size
            sizeDf2 = df2[columnValuesStr_PANI].size

            installation_date = pd.Timestamp(row["start_vent"])
            withdrawal_date = installation_date + pd.Timedelta(days=row["vent_lenght"])
            total_time_hour = (withdrawal_date - installation_date).total_seconds() / 3600 + 4

            liste_valeurs = []
            liste_mask = [] # permet de savoir si une valeur est manquante à l'origine

            if df[columnValuesStr_PAI].isnull().all() or sizeDf == 0:
                if df2[columnValuesStr_PANI].isnull().all() or sizeDf2 == 0:
                    liste_valeurs = [defaultValue] * int(total_time_hour)
                    liste_mask = [0] * int(total_time_hour)
                    sizeDf = 0
                else:
                    df = df2
                    columnValuesStr_PAI = columnValuesStr_PANI
                    sizeDf = sizeDf2

            if sizeDf > 0:
                for i in range(int(total_time_hour)):

                    if i >= sizeDf:
                        val_i = np.nan
                    else:
                        val_i = df[columnValuesStr_PAI][i]

                    if np.isnan(val_i):

                        if (i < sizeDf2) and (not(np.isnan(df2[columnValuesStr_PANI][i]))):
                            new_val = df2[columnValuesStr_PANI][i]
                            liste_valeurs.append(new_val)
                            liste_mask.append(1)
                        
                        else:
                            
                            liste_mask.append(0)

                            j = 1
                            found = False
                            
                            while not(found):
                                if i-j < 0:
                                    while i+j < sizeDf:
                                        val_i_plus_j = df[columnValuesStr_PAI][i+j]
                                        if not(np.isnan(val_i_plus_j)):
                                            new_val = val_i_plus_j
                                            found = True
                                            break
                                        j += 1
                                elif i+j >= sizeDf:
                                    while i-j >= sizeDf:
                                        j += 1
                                    while i-j >= 0:
                                        val_i_moins_j = df[columnValuesStr_PAI][i-j]
                                        if not(np.isnan(val_i_moins_j)):
                                            new_val = val_i_moins_j
                                            found = True
                                            break
                                        j += 1
                                else:
                                    val_i_moins_j = df[columnValuesStr_PAI][i-j]
                                    val_i_plus_j = df[columnValuesStr_PAI][i+j]
                                    if not(np.isnan(val_i_moins_j)):
                                        if not(np.isnan(val_i_plus_j)):
                                            new_val = (val_i_moins_j + val_i_plus_j) * 0.5
                                        else:
                                            new_val = val_i_moins_j
                                        found = True
                                    elif not(np.isnan(val_i_plus_j)):
                                        new_val = val_i_plus_j
                                        found = True
                                j += 1
                            
                            liste_valeurs.append(new_val)

                    else:
                        liste_mask.append(1)
                        liste_valeurs.append(val_i)
            

            newdf = pd.DataFrame({columnValuesStr: liste_valeurs})
            newDfPath = preProcessedDataPath + encounterId + "/" + columnValuesStr + "_Complet.parquet"
            pq.write_table(pa.Table.from_pandas(newdf), newDfPath)

            newdfMask = pd.DataFrame({'mask': liste_mask})
            newdfMaskPath = preProcessedDataPath + encounterId + "/" + columnValuesStr + "_Mask.parquet"
            pq.write_table(pa.Table.from_pandas(newdfMask), newdfMaskPath)

In [5]:
def allValues(variableStr, dataPath, preProcessedDataPath, patients_df):

    nb_patients = len(patients_df)
    
    arrayList = []

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):
        encounterId = str(row["stay_id"])
        dfPath = dataPath + preProcessedDataPath + encounterId + "/" + variableStr + "_Moy.parquet"
        df = pd.read_parquet(dfPath)
        values = df.to_numpy()[:,0]
        arrayList.append(values)

    tabValues = np.concatenate(arrayList)

    return tabValues

def exportStats(dataGroup):

    dataPath = "../dataMimic/"

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    nb_patients = len(patients_df)

    rawDataPath = "rawData/"
    preProcessedDataPath = "preProcessedData/"

    if dataGroup == "dataECMO":
        listeVar = ["HR", "SpO2", "PAD_I", "PAM_I", "PAS_I", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "DebitECMO"]
    else:
        listeVar = ["HR", "SpO2", "PAD_I", "PAM_I", "PAS_I", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2"]
    
    df_stats = pd.DataFrame()
    
    for variableStr in listeVar:
        if variableStr=="Temperature":
            variableStr = "Temperature_C"
            values = allValues(variableStr, dataPath, preProcessedDataPath, patients_df)
            variableStr = "Temperature"
        else:
            values = allValues(variableStr, dataPath, preProcessedDataPath, patients_df)
        mean = np.nanmean(values)
        sd = np.nanstd(values)
        mini = np.nanmin(values)
        maxi = np.nanmax(values)

        if variableStr.endswith("_I"):
            variableStr = variableStr[:-2]

        stat = pd.DataFrame({variableStr: [mean, sd, mini, maxi]})
        
        df_stats[variableStr] = stat.iloc[:, 0]

    df_stats_Path = dataPath + "stats.parquet"
    pq.write_table(pa.Table.from_pandas(df_stats), df_stats_Path)

In [18]:
def regroupement(dataGroup):

    if dataGroup == "dataECMO":
        dataPath = "dataECMO/"
        listeVar = ["HR", "SpO2", "PAD", "PAM", "PAS", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "Compliance", "DebitECMO"]    
    elif dataGroup == "dataRea":
        dataPath = "dataRea/"
        listeVar = ["HR", "SpO2", "PAD", "PAM", "PAS", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "Compliance"]
    else:
        dataPath = "../dataMimic/"
        listeVar = ["HR", "SpO2", "PAD", "PAM", "PAS", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "Compliance", "Dialyse"]

    patients_df = pd.read_csv(dataPath + "ventiles.csv")
    
    preProcessedDataPath = dataPath + "preProcessedData/"
    finalDataPath = dataPath + "finalData/"

    nb_patients = len(patients_df)
    
    df_stats_path = "../../dataRea/stats.parquet"
    df_stats = pd.read_parquet(df_stats_path)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        df_final = pd.DataFrame()
        
        
        for variableStr in listeVar:
            
            dfPath = preProcessedDataPath + encounterId + "/" + variableStr + "_Complet" + ".parquet"
            df = pd.read_parquet(dfPath)

            if variableStr == "Dialyse":
                df_final[variableStr] = df
            else:
                mean = df_stats[variableStr][0]
                sd = df_stats[variableStr][1]
                # mini = df_stats[variableStr][2]
                # maxi = df_stats[variableStr][3]
    
                if variableStr == "Diurese":
                    df_final[variableStr] = (df.iloc[:, 0]/1000 - mean) / sd
                else:
                    df_final[variableStr] = (df.iloc[:, 0] - mean) / sd
                # df_final[variableStr] = (df.iloc[:, 0] - mini) / (maxi - mini)


        df_final_Path = finalDataPath + encounterId + "/dynamic.parquet"
        pq.write_table(pa.Table.from_pandas(df_final), df_final_Path)


def regroupement_mask(dataGroup):

    if dataGroup == "dataECMO":
        dataPath = "dataECMO/"
        listeVar = ["HR", "SpO2", "PAD", "PAM", "PAS", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "Compliance", "DebitECMO"]    
    elif dataGroup == "dataRea":
        dataPath = "dataRea/"
        listeVar = ["HR", "SpO2", "PAD", "PAM", "PAS", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "Compliance"]
    else:
        dataPath = "../dataMimic/"
        listeVar = ["HR", "SpO2", "PAD", "PAM", "PAS", "RR", "Temperature", "Diurese", "SpO2_sur_FiO2", "FiO2", "Compliance", "Dialyse"]

    patients_df = pd.read_csv(dataPath + "ventiles.csv")

    preProcessedDataPath = dataPath + "preProcessedData/"
    finalDataPath = dataPath + "finalData/"

    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        df_final = pd.DataFrame()
        
        for variableStr in listeVar:

            dfPath = preProcessedDataPath + encounterId + "/" + variableStr + "_Mask" + ".parquet"
            df = pd.read_parquet(dfPath)
            df_final[variableStr] = df

        df_final_Path = finalDataPath + encounterId + "/mask.parquet"
        pq.write_table(pa.Table.from_pandas(df_final), df_final_Path)


def regroupement_static(dataGroup):
    
    if dataGroup == "dataECMO":
        dataPath = "dataECMO/"
    elif dataGroup == "dataRea":
        dataPath = "dataRea/"
    else:
        dataPath = "../dataMimic/"
        
    patients_df = pd.read_csv(dataPath + "ventiles.csv")
        
    preProcessedDataPath = dataPath + "preProcessedData/"
    finalDataPath = dataPath + "finalData/"

    nb_patients = len(patients_df)

    for index, row in tqdm(patients_df.iterrows(), total=nb_patients):

        encounterId = str(row["stay_id"])

        df_final_static = pd.DataFrame()

        age = int(patients_df.loc[patients_df["stay_id"]==int(encounterId),"Age"].to_numpy()[0])
        df_final_static['age'] = [(age - 18) / (93 - 18)]

        sexe = str(patients_df.loc[patients_df["stay_id"]==int(encounterId),"gender"].to_numpy()[0])
        if sexe == "M":
            df_final_static['sexe'] = [-1]
        else:
            df_final_static['sexe'] = [1]

        mean_IMC = 27.204   #Stats sur Réa Rangueil
        sd_IMC = 6.135
        min_IMC = 10.84
        max_IMC = 65.53
        dfPath = preProcessedDataPath + encounterId + "/IMC" + ".parquet"
        df = pd.read_parquet(dfPath)
        df_final_static['IMC'] = (df.iloc[:, 0] - mean_IMC) / sd_IMC
        # df_final_static['IMC'] = (df.iloc[:, 0] - min_IMC) / (max_IMC - min_IMC)


        df_final_Path = finalDataPath + encounterId + "/static.parquet"
        pq.write_table(pa.Table.from_pandas(df_final_static), df_final_Path)

In [19]:
def gestionDonneesAberrantesFull(dataGroup):

    # gestionDonneesAberrantes(dataGroup, "HR", 20, 200, 'HR')
    # gestionDonneesAberrantes(dataGroup, "SpO2", 50, 100, 'SpO2')
    # gestionDonneesAberrantes(dataGroup, "PAD_I", 20, 130, 'PAD_I')
    # gestionDonneesAberrantes(dataGroup, "PAD_NI", 20, 130, 'PAD_NI')
    # gestionDonneesAberrantes(dataGroup, "PAM_I", 30, 200, 'PAM_I')
    # gestionDonneesAberrantes(dataGroup, "PAM_NI", 30, 200, 'PAM_NI')
    # gestionDonneesAberrantes(dataGroup, "PAS_I", 40, 230, 'PAS_I')
    # gestionDonneesAberrantes(dataGroup, "PAS_NI", 40, 230, 'PAS_NI')
    # gestionDonneesAberrantes(dataGroup, "RR", 5, 50, 'RR')
    # gestionDonneesAberrantes(dataGroup, "Temperature_C", 32, 41, 'Temperature_C')
    # gestionDonneesAberrantes(dataGroup, "Temperature_F", 89.6, 105.8, 'Temperature_F')
    # gestionDonneesAberrantes(dataGroup, "Diurese", 0, 2000,'Diurese')
    # gestionDonneesAberrantes(dataGroup, "Weight", 30, 300, 'Weight')
    # gestionDonneesAberrantes(dataGroup, "Height", 120, 230, 'Height')
    # gestionDonneesAberrantes(dataGroup, "FiO2", 20, 100, 'FiO2')
    # gestionDonneesAberrantes(dataGroup, "Pplat", 5, 30, 'Pplat')
    # gestionDonneesAberrantes(dataGroup, "VolumeCourant", 100, 800, 'VolumeCourant')
    # gestionDonneesAberrantes(dataGroup, "PEP", 2, 22, 'PEP')
    
    if dataGroup == "dataECMO":
        gestionDonneesAberrantes(dataGroup, "DebitECMO", 0, 8, 'debit')


def moyennageFull(dataGroup):

    # gestionIMC(dataGroup)
    # gestionDiurese(dataGroup, 6)
    # moyenne_sur_x_minutes(dataGroup, "Diurese0", 60, "Diurese")
    # moyenne_sur_x_minutes(dataGroup, "HR", 60, "HR")
    # moyenne_sur_x_minutes(dataGroup, "SpO2", 60, "SpO2")
    # moyenne_sur_x_minutes(dataGroup, "PAD_I", 60, "PAD_I")
    # moyenne_sur_x_minutes(dataGroup, "PAM_I", 60, "PAM_I")
    # moyenne_sur_x_minutes(dataGroup, "PAS_I", 60, "PAS_I")
    # moyenne_sur_x_minutes(dataGroup, "RR", 60, "RR")
    # moyenne_sur_x_minutes(dataGroup, "Temperature_C", 60, "Temperature_C")
    # moyenne_sur_x_minutes(dataGroup, "Temperature_F", 60, "Temperature_F")
    # moyenne_sur_x_minutes(dataGroup, "PAD_NI", 60, 'PAD_NI')
    # moyenne_sur_x_minutes(dataGroup, "PAM_NI", 60, 'PAM_NI')
    # moyenne_sur_x_minutes(dataGroup, "PAS_NI", 60, 'PAS_NI')
    # moyenne_sur_x_minutes(dataGroup, "PAD_NI", 60, 'PAD_NI')
    # gestionFiO2(dataGroup, 60)

    # moyenne_sur_x_minutes(dataGroup, "FiO2", 60, 'FiO2') # Facultatif

    if dataGroup == "dataECMO":
        moyenne_sur_x_minutes(dataGroup, "DebitECMO", 60, "debit")


def gestionValeursManquantesFull(dataGroup):

    # gestionValeursManquantes(dataGroup, "HR", "HR", 85)
    # gestionValeursManquantes(dataGroup, "SpO2", "SpO2", 96)
    # gestionValeursManquantesPA(dataGroup, "PAD", "PAD", 60)
    # gestionValeursManquantesPA(dataGroup, "PAM", "PAM", 80)
    # gestionValeursManquantesPA(dataGroup, "PAS", "PAS", 125)
    # gestionValeursManquantes(dataGroup, "RR", "RR", 22)
    # gestionValeursManquantesTemperatures("Temperature", 37)
    # gestionValeursManquantes(dataGroup, "Diurese0", "Diurese", 0.0015, isDiurese=True)
    # gestionValeursManquantes(dataGroup, "SpO2_sur_FiO2", 'SpO2_sur_FiO2', 2.5)
    # gestionValeursManquantes(dataGroup, "Compliance", 'Compliance', 40)
    # gestionValeursManquantes(dataGroup, "FiO2", "FiO2", 40) # Facultatif

    if dataGroup == "dataECMO":
        gestionValeursManquantes(dataGroup, "DebitECMO", "debit", 3)


def exportStatsFull(dataGroup):
    exportStats(dataGroup)


def regroupementFull(dataGroup):
    regroupement(dataGroup)
    regroupement_mask(dataGroup)
    # regroupement_static(dataGroup)


# dataGroup = "dataECMO"
# dataGroup = "dataRea"
dataGroup = "dataMimic"

# gestionDonneesAberrantesFull(dataGroup)
# moyennageFull(dataGroup)
# gestionValeursManquantesFull(dataGroup)
# exportStatsFull(dataGroup)
regroupementFull(dataGroup)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4265/4265 [00:55<00:00, 77.40it/s]


In [7]:
dataGroup = "dataMimic"
# moyenne_sur_x_minutes(dataGroup, "PEP", 60, 'PEP')
moyenne_sur_x_minutes(dataGroup, "VolumeCourant", 60, 'VolumeCourant')
moyenne_sur_x_minutes(dataGroup, "Pplat", 60, 'Pplat')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4265/4265 [00:20<00:00, 206.99it/s]
